In [ ]:
# Author: Bronner 
# Date: 16.5.22

# Standard libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Time related libraries

from epiweeks import Week
from datetime import date

# Confidence interval

from statsmodels.stats.proportion import proportion_confint   

In [ ]:
# Functions to be used to check data 

def IQR_creation(data, name):
    """Create Median, Mean, IQR, Range for individual variables"""
    
    nb = len(data)
    missing_nb = sum(data.isnull())
    x50, x25, x75 = np.nanpercentile(data, [50, 25, 75])
    x_mean, x_min, x_max = data.mean(), data.min(), data.max()
    
    return pd.DataFrame({'Median': [x50], 'Mean' : [x_mean], 
                         'Q1' : [x25], 'Q3' : [x75], 
                         'Min' : [x_min], 'Max' : [x_max], 
                         'missing_nb' : [missing_nb], 
                         'nb' : [nb]}, index = [name])

def cat_table(data, list_var, list_name):
    """Create table with categorical variables 
    (adapted to the variable coding in the dataset for this project)"""
    
    data_new = pd.DataFrame()
    
    for var_idx, var_x in enumerate(list_var):
            positive = sum(data[var_x]>0)
            negative = sum(data[var_x]==0)
            
            if sum(data[var_x] < 0):
                error = 'Error'
            else:
                error = 'No Error'
            
            if (positive+negative) > 0:
                fraction_positive = positive/(positive+negative)
                
            elif (positive+negative) == 0:
                fraction_positive = np.nan

            is_missing = sum(data[var_x].isnull())
            
            assert sum(data[var_x].notnull()) == (positive+negative)
            
            data_new = data_new.append(pd.DataFrame({'positive': [positive], 'negative' : [negative],
                                                     'fraction_positive' : [fraction_positive], \
                                                     'missing_data' : [is_missing], 'error' : [error]}, \
                                                     index = [list_name[var_idx]]))
        
    return data_new

def missing_data_table(data):
    """Create a table with proportion of observations with missing data for each variable"""
    
    missing_data = pd.DataFrame((data.isnull().sum())/data.shape[0]).reset_index()
    missing_data.rename(columns = {'index' : 'Variables',
                                     0 : 'Proportion missing'}, inplace = True)
    return missing_data


In [ ]:
# Date today 

date_today = date.today()
print(date_today)

In [ ]:
# Current data 

data = pd.read_csv(data_file)
data

In [ ]:
# Names of variables

for var in data.columns:
    print(var)

In [ ]:
# Creating variables to define Year, Month, Day

print('Current variable types')
print(data[['admission.date', 'onset.date', 'outcome.date']].dtypes)

# Create old variables to double check
data['admission.date_OLD'] = data['admission.date'].copy()
data['outcome.date_OLD'] = data['outcome.date'].copy()

# Change type of date variables 
data['admission.date'] = pd.to_datetime(data['admission.date'])
data['onset.date'] =  pd.to_datetime(data['onset.date'])
data['outcome.date'] = pd.to_datetime(data['outcome.date'])

print('Variable types after correction')
print(data[['admission.date', 'onset.date', 'outcome.date']].dtypes)

# Create year/month/day variables
data['year_admission'] = data['admission.date'].dt.year.astype('int')
data['month_admission'] = data['admission.date'].dt.month.astype('int')
data['day_admission'] = data['admission.date'].dt.day.astype('int')

print('Number of observations with missing admission date')
print(sum(data['admission.date'].isnull()))

data[['admission.date', 'admission.date_OLD', 'outcome.date', 'outcome.date_OLD', 'onset.date', 'year_admission', 'month_admission', 'day_admission']].head(50)


In [ ]:
# Create table with total number of observations per country (before removing observations outside study period)

current_table_title = 'table_1_' + str(date_today) + '.csv' 
current_table = pd.DataFrame(data['Country'].value_counts())
current_table.to_csv(current_table_title)
current_table

In [ ]:
# Date range

print('Range of dates in current data')
print(data['admission.date'].min())
print(data['admission.date'].max())

In [ ]:
# Number of countries

print(data.loc[data['Country'].notnull(), 'Country'].nunique())
print(sorted(data.loc[data['Country'].notnull(), 'Country'].unique()))

In [ ]:
## Removing observations with missing country

print('Number of observations with missing country')
print(data.loc[data['Country'].isnull(), : ].shape)

print('Shape before')
print(data.shape)
data = data.loc[data['Country'].notnull(), : ].copy().reset_index(drop = True)
print('Shape after')
print(data.shape)
data

In [ ]:
# Number of records per country (1/2)

print('Shape before')
print(data.shape)
data['Zero_Country'] = data.groupby('Country').cumcount()
data['Total_Country'] = data.groupby('Country')['Country'].transform(lambda x : x.count())
print('Shape after')
print(data.shape)

data[['Zero_Country', 'Total_Country', 'Country']].head(50)

In [ ]:
# Number of records per country (2/2)

print('Approach 1\n')
print(data.loc[data['Zero_Country'] == 0, ['Country' , 'Total_Country']])

print('Approach 2\n')
print(data['Country'].value_counts())

assert data.shape[0] == data['Country'].value_counts().sum()

In [ ]:
# Median / IQR observations per country

print('Median (IQR) observations per country')

print(np.percentile(data['Country'].value_counts().values, [50, 25, 75]))
print(np.percentile(data.loc[data['Zero_Country'] ==0 , 'Total_Country'].values, [50, 25, 75]))

In [ ]:
# % of observations from the UK and South Africa

data_nb_per_country = pd.DataFrame(data['Country'].value_counts())

print('Number of observations per country')
print(data_nb_per_country)

print('\n')
print(data_nb_per_country.sum().values[0])

print('\n')
print('South Africa')
print('Absolute numbers')
print(data_nb_per_country.loc['South Africa'].values[0])
print('Proportion')
print(100*data_nb_per_country.loc['South Africa'].values[0]/data_nb_per_country.sum().values[0])

print('UK')
print('Absolute numbers')
print(data_nb_per_country.loc['UK'].values[0])
print('Proportion')
print(100*data_nb_per_country.loc['UK'].values[0]/data_nb_per_country.sum().values[0])

In [ ]:
# Number of observations outside UK & SA

print('Number of observations outside UK & SA')
data.loc[(data['Country'] != "South Africa") & (data['Country'] != "UK"), : ].shape[0]

In [ ]:
# List of symptoms and comorbidities

list_symptoms_overall = ['cough.any', 'fever', 'headache', 'confusion', \
                        'seizures', 'sore.throat', 'runny.nose', 'vomiting', \
                         'wheezing', 'diarrhoea', 'chest.pain', 'conjunctivitis', \
                         'myalgia', 'rash','fatigue', 'ageusia',\
                         'inability.to.walk', 'anosmia', 'shortness.of.breath','lymphadenopathy']

list_comorbidities_overall = ['liver.disease', 'diabetes', \
                              'chronic.cardiac.disease', 'hypertension', \
                              'current.smoking', 'copd', \
                              'active.tb', 'asthma', \
                              'chronic.kidney.disease', 'malignant.neoplasm',\
                              'dementia', 'hiv.aids', \
                              'obesity', 'chronic.neurological.disorder']


In [ ]:
# Missing symptoms

current_table_title = 'missing_data_symptoms_overall' + str(date_today) + '.csv' 

current_table = cat_table(data.loc[data['Country'] != 'South Africa', : ].copy(), list_symptoms_overall,\
                                                                                  list_symptoms_overall).reset_index()

current_table.rename(columns = {'index' : 'symptoms'}, inplace = True)
current_table.to_csv(current_table_title)

current_table


In [ ]:
# Missing comorbidities

current_table_title = 'missing_data_comorbidities_overall' + str(date_today) + '.csv' 

current_table = cat_table(data, list_comorbidities_overall,list_comorbidities_overall).reset_index()

current_table.rename(columns = {'index' : 'comorbidities'}, inplace = True)
current_table.to_csv(current_table_title)

current_table


In [ ]:
## Creating variables related to start of follow-up and follow-up duration (1/2)

data['date_start'] = data['admission.date'].copy()
data.loc[(data['admission.date'] < data['onset.date']), 'date_start'] = data['onset.date']
data['date_start'] = pd.to_datetime(data['date_start'])

print('Number of missing start dates')
print(sum(data['date_start'].isnull()))
print('\n\n')

print('Observations with missing start date')
print(data.loc[(data['date_start'].isnull()), ['date_start', 'admission.date', 'onset.date']].head(10))

print('Observations to compare')
data[['date_start', 'admission.date', 'onset.date']].head(50)


In [ ]:
## Creating variables related to start of follow-up and follow-up duration (2/2)

data['duration_fu'] = (data['outcome.date'] - data['date_start']).dt.days
print('Description duration_fu')
print(data['duration_fu'].describe())
print('\n\n')

print('Number of observations with duration_fu negative')
print(data.loc[data['duration_fu'] < 0, : ].shape[0])
print('\n\n')

print('\n\n')
print('Modifying data so that duration negative is missing')
data.loc[data['duration_fu'] < 0, 'duration_fu'] = np.nan


In [ ]:
# Creating age categories

bins = pd.IntervalIndex.from_tuples([(-1, 10), (10, 20), (20, 30), \
                                     (30, 40), (40, 50), (50, 60), \
                                     (60, 70), (70, 80), (80, 130)])

data['agegp10_raw'] = pd.cut(data['age'].values, bins)
data['agegp10'] = data['agegp10_raw'].values.codes

print('Bins used')
print(bins)
print('\n\n')

print('Number of numerical age groups')
print(data['agegp10'].unique())
print('\n\n')

print('Checking if missing data in categories')
print(data.loc[data['agegp10'].isnull(), : ].shape[0])
print(data.loc[data['agegp10'] == -1, : ].shape[0])
print(data.loc[data['agegp10'] == -1, ['age', 'agegp10']])
print('\n\n')

print('Correcting if needed for missing age categories')
data.loc[data['agegp10'] == -1, 'agegp10'] = np.nan
print('\n\n')

data[['age', 'agegp10_raw', 'agegp10']].head(50)

In [ ]:
## Defining highest ranking symptoms (not including data from South Africa)

symptoms_data = cat_table(data.loc[data['Country'] != 'South Africa', : ].copy(), \
                          list_symptoms_overall, list_symptoms_overall).reset_index()

symptoms_data = symptoms_data.sort_values(by = 'fraction_positive').reset_index(drop = True)
symptoms_data


In [ ]:
list_symptoms = (list(symptoms_data['index'][-5:].values))
list_symptoms.reverse()
list_symptoms

In [ ]:
## Creating numerical variables 

data['cough_num'] = np.nan
data.loc[data['cough.any'] == True, 'cough_num'] = 1
data.loc[data['cough.any'] == False, 'cough_num'] = 0

data['SOB_num'] = np.nan
data.loc[data['shortness.of.breath'] == True, 'SOB_num'] = 1
data.loc[data['shortness.of.breath'] == False, 'SOB_num'] = 0

data['fever_num'] = np.nan
data.loc[data['fever'] == True, 'fever_num'] = 1
data.loc[data['fever'] == False, 'fever_num'] = 0

data['fatigue_num'] = np.nan
data.loc[data['fatigue'] == True, 'fatigue_num'] = 1
data.loc[data['fatigue'] == False, 'fatigue_num'] = 0

data['vomiting_num'] = np.nan
data.loc[data['vomiting'] == True, 'vomiting_num'] = 1
data.loc[data['vomiting'] == False, 'vomiting_num'] = 0


In [ ]:
# Checking numerical variable 

print('Original variable')
print(data['shortness.of.breath'].value_counts())
print('\n\n')

print('New variable')
print(data['SOB_num'].value_counts())
print('\n\n')

print('Table with original and new variagbles')
print(pd.crosstab(data['shortness.of.breath'], data['SOB_num']))
print('\n\n')


In [ ]:
## Defining highest ranking comorbidities

comorbidities_data = cat_table(data.copy(), list_comorbidities_overall, \
                               list_comorbidities_overall).reset_index()


comorbidities_data = comorbidities_data.sort_values(by = 'fraction_positive').reset_index(drop = True)
comorbidities_data


In [ ]:
list_comorbidities = (list(comorbidities_data['index'][-5:].values))
list_comorbidities.reverse()
list_comorbidities

In [ ]:
## Creating numeric variables

data['hypertension_num'] = np.nan
data.loc[data['hypertension'] == True, 'hypertension_num'] = 1
data.loc[data['hypertension'] == False, 'hypertension_num'] = 0

data['smoking_num'] = np.nan
data.loc[data['current.smoking'] == True, 'smoking_num'] = 1
data.loc[data['current.smoking'] == False, 'smoking_num'] = 0

data['diabetes_num'] = np.nan
data.loc[data['diabetes'] == True, 'diabetes_num'] = 1
data.loc[data['diabetes'] == False, 'diabetes_num'] = 0

data['cardiac_num'] = np.nan
data.loc[data['chronic.cardiac.disease'] == True, 'cardiac_num'] = 1
data.loc[data['chronic.cardiac.disease'] == False, 'cardiac_num'] = 0

data['neurological_num'] = np.nan
data.loc[data['chronic.neurological.disorder'] == True, 'neurological_num'] = 1
data.loc[data['chronic.neurological.disorder'] == False, 'neurological_num'] = 0


In [ ]:
# Checking numerical variable 

print('Original variable')
print(data['diabetes'].value_counts())
print('\n\n')
print('New variable')
print(data['diabetes_num'].value_counts())
print('\n\n')
print('Table with original and new variagbles')
print(pd.crosstab(data['diabetes'], data['diabetes_num']))
print('\n\n')

In [ ]:
## Creating variable with at least one comorbidity

print('Shape before')
print(data.shape)
data['at_least_1_comorbidity'] = data[list_comorbidities_overall].max(axis = 1)

data['nb_comorbidity_not_missing'] = data[list_comorbidities_overall].copy().notnull().sum(axis = 1)
print('Shape after')
print(data.shape)

list_to_check = list_comorbidities_overall + ['at_least_1_comorbidity', 'nb_comorbidity_not_missing', 'Country']
data[list_to_check].head(30)

In [ ]:
data.loc[data['at_least_1_comorbidity'].isnull(), list_to_check]

In [ ]:
## Variable with at least one symptom
pd.options.display.max_columns = 50

print('Shape before')
print(data.shape)

data['at_least_1_symptom'] = data[list_symptoms_overall].max(1)
data['nb_symptoms_not_missing'] = data[list_symptoms_overall].copy().notnull().sum(1)

print('Shape after')
print(data.shape)

list_to_check = list_symptoms_overall + ['at_least_1_symptom', 'nb_symptoms_not_missing']
data[list_to_check].head(30)

In [ ]:
### ### ### ### ### ### 
## Population-level data 

variant_timing_data = pd.read_csv(data_with_threshold_dates_per_country)

print('Shape')
print(variant_timing_data.shape)

variant_timing_data.head(10)


In [ ]:
## Merging variant dataset and clinical data

data = pd.merge(variant_timing_data, data, on = 'iso3', indicator = True, how = 'outer')

print('Looking merge of clinical data and variant data')
print(data['_merge'].value_counts())

print(data.loc[data['_merge'] == 'left_only', 'iso3' ].unique())
print(data.loc[data['_merge'] == 'right_only', 'iso3' ].unique())

data

In [ ]:
## Complete merging by removing unused rows

assert data.loc[data['_merge'] == 'right_only', : ].shape[0] == 0
data = data.loc[data['_merge'] == 'both', : ].copy().reset_index(drop = True)
del data['_merge']
data

In [ ]:
## Creating variables corresponding to study periods
time_window_days = 60

data['time_periods'] = np.nan
data.loc[((data['admission.date'] -  pd.to_datetime(data['var_above'])).dt.days >= 0) & \
         ((data['admission.date'] -  pd.to_datetime(data['var_above'])).dt.days < time_window_days), 'time_periods' ] = 1

data.loc[((pd.to_datetime(data['var_below']) - data['admission.date']).dt.days >= 0) & \
         ((pd.to_datetime(data['var_below']) - data['admission.date']).dt.days < time_window_days), 'time_periods' ] = 0

print('Table of time_periods')
print(data['time_periods'].value_counts())
print('\n\n')

print('Checking if variable was created correctly')
data[['time_periods', 'whoname', 'admission.date', 'var_below', 'var_above']].head(50)


In [ ]:
## Descriptions to be included in the paper

print('Number of observations in either study periods')
print(data.loc[data['time_periods'].notnull(), : ].shape)
print('\n\n')

print('Number of observations in each study period')
print(data.loc[data['time_periods'].notnull(), 'time_periods'].value_counts())
print('\n\n')

print('Number of countries with observations within study periods')
print(data.loc[data['time_periods'].notnull(), 'Country'].nunique())
print('\n\n')



In [ ]:
### Creating a dataset that only includes observations during study periods

print('Shape before')
print(data.shape)
data_Omicron = data.loc[data['time_periods'].notnull(), : ].copy()
print('Shape after')
print(data_Omicron.shape)
data_Omicron


In [ ]:
## Description for the dataset with observations during study periods only

print('Distribution of observations by time periods')
print(data_Omicron['time_periods'].value_counts())

print('Number of countries')
print(data['Country'].nunique())
print(data_Omicron['Country'].nunique())
print('\n\n')

print('List of unique countries in each dataset')
print(sorted(list(data['Country'].unique())))
print('\n')
print(sorted(list(data_Omicron['Country'].unique())))

In [ ]:
## Previous COVID variable

print('Number of observations with information on Previous COVID')
print(sum(data_Omicron['previous.covid'].notnull()))
print('\n')

print('Distribution of observations by Previous COVID')
print(data_Omicron['previous.covid'].value_counts())
print('\n')

print('Tabulate distribution of previous COVID by time period')
print(pd.crosstab(data_Omicron['time_periods'], data_Omicron['previous.covid']))
print('\n')

print('% with previous COVID in pre-Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 0, 'previous.covid'].value_counts().loc[1])
print(data_Omicron.loc[data_Omicron['time_periods'] == 0, 'previous.covid'].value_counts().loc[1]/(data_Omicron.loc[data_Omicron['time_periods'] == 0, 'previous.covid'].value_counts().sum()))
print('\n')
      
print('% with previous COVID in Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 1, 'previous.covid'].value_counts().loc[1])
print(data_Omicron.loc[data_Omicron['time_periods'] == 1, 'previous.covid'].value_counts().loc[1]/(data_Omicron.loc[data_Omicron['time_periods'] == 1, 'previous.covid'].value_counts().sum()))
print('\n')

print('Distribution of Previous COVID by country')
print(pd.crosstab(data_Omicron['Country'], data_Omicron['previous.covid']))
      

In [ ]:
## Presenting distribution of age and sex by study period


print('Distribution of age by study period')
print('\n')

print('Age - pre-Omicron period')
print(IQR_creation(data_Omicron.loc[data_Omicron['time_periods'] == 0 , 'age'], 'Age'))
print('\n')

print('Age - Omicron period')
print(IQR_creation(data_Omicron.loc[data_Omicron['time_periods'] == 1 , 'age'], 'Age'))
print('\n\n')

print('Distribution of sex by study period')
print('\n')

print('Sex - pre-Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 0, 'birth.sex'].value_counts())
print(data_Omicron.loc[(data_Omicron['time_periods'] == 0) & \
                       (data_Omicron['birth.sex'] != 'Unknown'), 'birth.sex'].value_counts(normalize = True))
print('\n')

print('Sex - Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 1, 'birth.sex'].value_counts())
print(data_Omicron.loc[(data_Omicron['time_periods'] == 1) & \
                       (data_Omicron['birth.sex'] != 'Unknown'), 'birth.sex'].value_counts(normalize = True))
print('\n')

In [ ]:
## Variable on the reason for admission

print('Total number of observations pre-Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 0, 'covid.reason.for.admission'].value_counts().sum())
print('\n')
print('Distribution of observations by reason during pre-Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 0, 'covid.reason.for.admission'].value_counts())
print('\n')
print(data_Omicron.loc[(data_Omicron['time_periods'] == 0), 'covid.reason.for.admission'].value_counts(normalize = True))
print('\n\n')

print('Total number of observations Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 1, 'covid.reason.for.admission'].value_counts().sum())
print('\n')
print('Distribution of observations by reason during Omicron period')
print(data_Omicron.loc[data_Omicron['time_periods'] == 1, 'covid.reason.for.admission'].value_counts())
print('\n')
print(data_Omicron.loc[(data_Omicron['time_periods'] == 1), 'covid.reason.for.admission'].value_counts(normalize = True))
print('\n')

print('Countries with information on reason for admission')
print(data_Omicron.loc[(data_Omicron['covid.reason.for.admission'].notnull()), 'Country'].value_counts())
print('\n')

print('% of observations with information on reason from South Africa')
print((data_Omicron.loc[(data_Omicron['covid.reason.for.admission'].notnull()), \
                        'Country'].value_counts().loc['South Africa'])/(data_Omicron.loc[(data_Omicron['covid.reason.for.admission'].notnull()), 'Country'].value_counts().sum()))
print('\n')



In [ ]:
current_table_title = 'numbers_by_country_and_period' + str(date_today) + '.csv' 

current_table = pd.crosstab(data_Omicron['Country'], data_Omicron['time_periods'])
current_table['sum'] = current_table.sum(axis = 1)
current_table = current_table.sort_values(by = 'sum', ascending = False)
current_table.to_csv(current_table_title)
current_table

In [ ]:
## Information to be included in the Methods section 

print('Number of total observations in the study periods')
print(data_Omicron.shape)
print('\n')
print('Number of total observations with missing admission date in the study periods')
print(sum(data_Omicron['admission.date'].isnull()))
print('\n')
print('Number of total observations with missing onset date in the study periods')
print(sum(data_Omicron['onset.date'].isnull()))
print('\n')
print('% of total observations with missing onset date in the study periods')
print(sum(data_Omicron['onset.date'].isnull())/data_Omicron.shape[0])
print('\n')
print('\n')

print('Number of observations with missing outcome date')
print(sum(data_Omicron['outcome.date'].isnull()))
print('\n')
print('% of observations with missing outcome date')
print(sum(data_Omicron['outcome.date'].isnull())/data_Omicron.shape[0])
print('\n')

mask_now = (data_Omicron['outcome.date'].notnull()) & (data_Omicron['date_start'].notnull())

print('Number of observations with non-missing outcome date and non-missing start date, and outcome before start')
print(data_Omicron.loc[(mask_now) & (data_Omicron['outcome.date'] < data_Omicron['date_start']), : ].shape[0])
print('\n')
print('% of observations with non-missing outcome date and non-missing start date, and outcome before start')
print((data_Omicron.loc[(mask_now) & (data_Omicron['outcome.date'] < data_Omicron['date_start']), : ].shape[0])/data_Omicron.shape[0])
print('\n')


In [ ]:
# Create table of median (IQR) age per country

Data_age_country = pd.DataFrame()

for country in list(data_Omicron['Country'].unique()):
    print(country)
    pre_data = data_Omicron.loc[(data_Omicron['Country'] == country) & \
                                (data_Omicron['time_periods'] == 0), : ].copy()
    post_data = data_Omicron.loc[(data_Omicron['Country'] == country) & \
                                (data_Omicron['time_periods'] == 1), : ].copy()    
    
    if (pre_data.shape[0] < 10) or (post_data.shape[0] < 10):
        print('Continue because less than 10 in at least one period')
        continue
        
    x25e, x50e, x75e = np.nanpercentile(pre_data['age'], [25, 50, 75])
    x25o, x50o, x75o = np.nanpercentile(post_data['age'], [25, 50, 75])    
    
    Data_age_country = Data_age_country.append(pd.DataFrame({'country' : [country],
                                                            'x25e' : [x25e], 'x50e' : [x50e], 'x75e' : [x75e],
                                                            'x25o' : [x25o], 'x50o' : [x50o], 'x75o' : [x75o]}))
Data_age_country

In [ ]:
current_table_title = 'data_age_country' + str(date_today) + '.csv' 

Data_age_country.to_csv(current_table_title)
Data_age_country

In [ ]:
## Description of "at least one comorbidity"

print('Number of non-missing observations "at least 1 comorbidity"')
print(sum(data_Omicron['at_least_1_comorbidity'].notnull()))
print('\n')

print('Number of missing observations "at least 1 comorbidity"')
print(sum(data_Omicron['at_least_1_comorbidity'].isnull()))
print('\n')

print('Frequency of at 1 least comorbidity by period')
print(pd.crosstab(data_Omicron['time_periods'], data_Omicron['at_least_1_comorbidity']))
print('\n\n\n')


print('Number of non-missing comorbidities')
print(np.nanpercentile(data_Omicron.loc[data_Omicron['time_periods'] == 0, \
                                        'nb_comorbidity_not_missing'].values, [25, 50, 75]))
print(np.nanpercentile(data_Omicron.loc[data_Omicron['time_periods'] == 1, \
                                        'nb_comorbidity_not_missing'].values, [25, 50, 75]))


print('Total number of comorbidities')
print(len(list_comorbidities_overall))


In [ ]:
## Description of "at least one symptom"

print('Number of non-missing observations "at least 1 symptom"')
print(sum(data_Omicron.loc[(data_Omicron['Country'] != 'South Africa'), 'at_least_1_symptom'].notnull()))
print('\n')
print('Number of missing observations "at least 1 symptom"')
print(sum(data_Omicron.loc[(data_Omicron['Country'] != 'South Africa'), 'at_least_1_symptom'].isnull()))
print('\n')
print('Number of missing observations "at least 1 symptom" by country')
print(data_Omicron.loc[(data_Omicron['at_least_1_symptom'].isnull()) & \
                       (data_Omicron['Country'] != 'South Africa'), 'Country'].value_counts())

print('\n\n\n')

print('Frequency of at 1 least symptom by period')
print(pd.crosstab(data_Omicron.loc[(data_Omicron['Country'] != 'South Africa'), 'time_periods'], \
                  data_Omicron.loc[(data_Omicron['Country'] != 'South Africa'),'at_least_1_symptom']))

print('\n\n\n')

print('Number of non-missing symptoms')
print(np.nanpercentile(data_Omicron.loc[(data_Omicron['time_periods'] == 0) & \
                                        (data_Omicron['Country'] != 'South Africa'), \
                                        'nb_symptoms_not_missing'].values, [25, 50, 75]))
print(np.nanpercentile(data_Omicron.loc[(data_Omicron['time_periods'] == 1) & \
                                        (data_Omicron['Country'] != 'South Africa'), \
                                        'nb_symptoms_not_missing'].values, [25, 50, 75]))

print('\n\n\n')


In [ ]:
## Descriptive information - death outcome

print('Number of deaths')
print(data_Omicron['outcome'].value_counts())
print('Distribution outcome_14_num')
print(data_Omicron['outcome_14_num'].value_counts())
print('Total non-missing outcome_14_num')
print(data_Omicron['outcome_14_num'].value_counts().sum())
print('\n\n')

print('Distribution outcome_28_num')
print(data_Omicron['outcome_28_num'].value_counts())
print('Total non-missing outcome_28_num')
print(data_Omicron['outcome_28_num'].value_counts().sum())
print('\n\n')

print('Number of deaths after day 28')
print(data_Omicron.loc[(data_Omicron['outcome'] == 'death') & \
                       (data_Omicron['duration_fu'] > 28) , : ].shape)


print('Number of deaths with missing duration_fu')
print(data_Omicron.loc[(data_Omicron['outcome'] == 'death') & \
                       (data_Omicron['duration_fu'].isnull()) , : ].shape)


In [ ]:
## Reporting time to death and time to recovery 

print('Duration of follow-up for patients who died during the pre-Omicron period')
print(data_Omicron.loc[(data_Omicron['time_periods'] == 0) & \
                       (data_Omicron['duration_fu'] >=0) & \
                       (data_Omicron['outcome'] == 'death' ), 'duration_fu'].describe())
print('\n')


print('Duration of follow-up for patients who died during the Omicron period')
print(data_Omicron.loc[(data_Omicron['time_periods'] == 1) & \
                       (data_Omicron['duration_fu'] >=0) & \
                       (data_Omicron['outcome'] == 'death' ), 'duration_fu'].describe())

print('\n')


In [ ]:
## Individual-level data

print('Number of observations with variant information')
print(data.loc[data['variant'].notnull(), : ].shape)
print('Number of countries with variant information')
print(data.loc[data['variant'].notnull(), 'Country'].nunique())
print('Number of observations per country with variant information')
print(data.loc[data['variant'].notnull(), 'Country'].value_counts())
print('\n\n')

print('Number of observations with variant information during either study period')
print(data.loc[(data['variant'].notnull()) & (data['time_periods'].notnull()), : ].shape)
print('\n')

print('Tabulation of variant and country -- pre-Omicron period')
print(pd.crosstab(data.loc[(data['variant'].notnull()) \
                     & (data['time_periods'].notnull()) &\
                     (data['time_periods'] == 0), 'Country'], data.loc[(data['variant'].notnull()) & \
                                                                       (data['time_periods'].notnull()) &\
                                                                       (data['time_periods'] == 0), 'variant']))

print('Tabulation of variant and country -- Omicron period')
print(pd.crosstab(data.loc[(data['variant'].notnull()) \
                     & (data['time_periods'].notnull()) &\
                     (data['time_periods'] == 1), 'Country'], data.loc[(data['variant'].notnull()) & \
                                                                       (data['time_periods'].notnull()) &\
                                                                       (data['time_periods'] == 1), 'variant']))



In [ ]:
## Figures

In [ ]:

%matplotlib qt
from matplotlib.lines import Line2D


Figure_A = plt.figure(figsize = (8, 10))

comorbidities_or_symptoms = str(input('Comorbidities (C) or Symptoms (S)?'))

if comorbidities_or_symptoms == 'C':
    title_list = list_comorbidities_names
    list_variables = list_comorbidities
    
elif comorbidities_or_symptoms == 'S':   
    title_list = list_symptoms_names
    list_variables = list_symptoms

print('List of titles')
print(title_list)
print('List of variables')
print(list_variables)

list_colors = ['lightskyblue', 'indianred']
print('List of colors')
print(list_colors)

nb_rows = len(list_variables)
nb_cols = 1
ax_list = []
for ax_idx in range(len(list_variables)):
    ax_list.append(Figure_A.add_subplot(nb_rows, nb_cols, (ax_idx + 1)))

xticks = []
xticklabels = []

p = 1
for country in sorted(list(data_Omicron['Country'].unique())):
    print(country)
    
    if (country == 'South Africa') & (comorbidities_or_symptoms == 'S'):
        continue
        print('Skipping South Africa for the symptoms')    
    
    print('Creating datasets that are country and time period specific')
    
    temp_data_0 = data_Omicron.loc[(data_Omicron['Country'] == country) & \
                                   (data_Omicron['time_periods'] == 0), : ].copy()

    temp_data_1 = data_Omicron.loc[(data_Omicron['Country'] == country) & \
                                   (data_Omicron['time_periods'] == 1), : ].copy()
    
    print('Number of observations pre-Omicron (overall)')
    n_0 = temp_data_0.shape[0]
    print(n_0)
    print('Number of observations Omicron (overall)')    
    n_1 = temp_data_1.shape[0]   
    print(n_1)
    
    if (n_0<10) or (n_1<10):
        print('Continue if less than 10 observations in one of the periods')
        continue
    
    # X data
    x_0 = p - 0.3
    x_1 = p + 0.3    
    
    print('X-axis coordinates - pre-Omicron and Omicron')
    print(x_0, x_1)
    
    ## Loop over list of variables
    
    for variable_idx, current_variable in enumerate(list_variables):
    
        temp_data_0_var = pd.DataFrame(temp_data_0[current_variable].value_counts(normalize = True)).sort_index()
        temp_data_1_var = pd.DataFrame(temp_data_1[current_variable].value_counts(normalize = True)).sort_index()    
        
        print('Current variable')
        print(current_variable)
        
        current_ax = ax_list[variable_idx]
        
        ## Criteria to plot zero proportions
        if 1 in temp_data_0_var.index:
            y_var_0 = temp_data_0_var.loc[1].values[0]
            
        else:
            y_var_0 = 0
                
        if 1 in temp_data_1_var.index:            
            y_var_1 = temp_data_1_var.loc[1].values[0]
            
        else:
            y_var_1 = 0

        ## Absolute counts of positive 
        if 1 in temp_data_0_var.index:
            num_var_0 = pd.DataFrame(temp_data_0[current_variable].value_counts()).sort_index().loc[1].values[0]
        else:
            num_var_0 = 0
                
        if 1 in temp_data_1_var.index:            
            num_var_1 = pd.DataFrame(temp_data_1[current_variable].value_counts()).sort_index().loc[1].values[0]
        else:
            num_var_1 = 0          

        # Total number of observations 
        total_0_var = pd.DataFrame(temp_data_0[current_variable].value_counts()).sort_index().sum()[0]
        total_1_var = pd.DataFrame(temp_data_1[current_variable].value_counts()).sort_index().sum()[0]   
            
        print('Temporary data - pre-Omicron') 
        print(temp_data_0_var)
        if total_0_var>0:
            print(y_var_0, num_var_0, num_var_0/total_0_var)
        else: 
            print(y_var_0, num_var_0)            
            
        print('Temporary data - Omicron') 
        print(temp_data_1_var)
        
        if total_1_var>0:
            print(y_var_1, num_var_1, num_var_1/total_1_var)
        else: 
            print(y_var_1, num_var_1)            
        
        print('\n\n')
        
        if total_0_var >= 5: 
            current_ax.bar(x_0, y_var_0, color = list_colors[0], \
                                               width = 0.6, edgecolor = 'k', bottom = 0)
            
            lower_0_limit, upper_0_limit = proportion_confint(num_var_0, total_0_var, alpha=0.05, method='beta')
            current_ax.plot([x_0, x_0], [lower_0_limit, upper_0_limit], color = 'k', linewidth = 1)
            
        elif total_0_var < 5:
            current_ax.text(x_0, 0.05, 'NS', fontsize = 4, fontweight = 'bold', horizontalalignment = 'center')
            
            
        if total_1_var >=5:            
            current_ax.bar(x_1, y_var_1, color = list_colors[1], \
                                               width = 0.6, edgecolor = 'k', bottom = 0)

            lower_1_limit, upper_1_limit = proportion_confint(num_var_1, total_1_var, alpha=0.05, method='beta')
            current_ax.plot([x_1, x_1], [lower_1_limit, upper_1_limit], color = 'k', linewidth = 1)        
                        
        elif total_1_var<5:
            current_ax.text(x_1, 0.05, 'NS', fontsize = 4, fontweight = 'bold', horizontalalignment = 'center')            
    
    xticks.append(p)
    country_label = country
        
    if country_label == "United States of America":
        country_label = 'USA'

    if country_label == "United Kingdom":
        country_label = 'UK'
            
    xticklabels.append(country_label)

    p += 2

for current_idx, current_ax in enumerate(ax_list):
    current_ax.set_xticks(xticks)
    current_ax.set_xticklabels(xticklabels, fontsize = 8)
    current_ax.set_title(title_list[current_idx], fontweight = 'bold')
    current_ax.set_ylim([0,1])
    current_ax.set_xlim(xmin = -0.3)    
    current_ax.set_ylabel('Proportion', fontweight = 'bold')
    
    
Figure_A.tight_layout()



In [ ]:

from matplotlib.lines import Line2D
Figure_B = plt.figure(figsize = (10, 10))

print('List of colors')
list_colors = ['lightskyblue', 'indianred']
print(list_colors)
print('\n')

print('Age categories')
age_list = [[-1, 18], [18, 60], [60, 130]]
age_ticks = ['<18', '18-60', '>60']
print(age_list)
print(age_ticks)
print('\n')

ax_list = []
for ax_idx in range(len(list_countries)):
    ax_list.append(Figure_B.add_subplot(nb_rows, nb_cols, (ax_idx + 1)))

for country_idx, country in enumerate(list_countries):
    print(country)

    current_ax = ax_list[country_idx]

    print('Generating temporary datasets by country and time period')
    temp_data_0 = data_Omicron.loc[(data_Omicron['Country'] == country) & \
                                    (data_Omicron['time_periods'] == 0), : ].copy()

    temp_data_1 = data_Omicron.loc[(data_Omicron['Country'] == country) & \
                                   (data_Omicron['time_periods'] == 1), : ].copy()
    
    print('Number of observation by country and study period')
    n_0 = temp_data_0.shape[0]
    n_1 = temp_data_1.shape[0]
    
    print('Number pre-Omicron: ',n_0, ' , Number Omicron: ',  n_1)
    
    if (n_0 <10) or (n_1 < 10):
        print('Continue because less than 10 observations either before or during Omicron')
        continue

    xticks = []
    xticklabels = []
    p = 1

    for age_idx, age_pair in enumerate(age_list):
        print(age_pair)
        
        print('Creating temporary datasets with age specific information and non-missing vaccination data')
        
        age_temp_data_0 = temp_data_0.loc[(temp_data_0['age'] >= age_pair[0]) & \
                                          (temp_data_0['age'] < age_pair[1]) & \
                                          (temp_data_0['vaccinated'].notnull()), : ].copy()
        
        age_temp_data_1 = temp_data_1.loc[(temp_data_1['age'] >= age_pair[0]) & \
                                          (temp_data_1['age'] < age_pair[1]) & \
                                          (temp_data_1['vaccinated'].notnull()), : ].copy()

        x_0 = p-0.3
        x_1 = p+0.3
        
        print('X-axis for age group pre-Omicron: ', x_0)
        print('X-axis for age group Omicron: ', x_1)        
        
        denominator_0 = age_temp_data_0.shape[0]
        denominator_1 = age_temp_data_1.shape[0]        
    
        print('Number of observations non-missing vaccination pre-Omicron')
        print(denominator_0)
        print('Number of observations non-missing vaccination Omicron')        
        print(denominator_1)
        
        numerator_0 = age_temp_data_0.loc[age_temp_data_0['vaccinated'] == True, : ].shape[0]
        numerator_1 = age_temp_data_1.loc[age_temp_data_1['vaccinated'] == True, : ].shape[0]        
    
        if denominator_0 > 0:
            y_0 = numerator_0/denominator_0
        else:
            y_0 = 0
    
        if denominator_1 > 0:
            y_1 = numerator_1/denominator_1
        else:
            y_1 = 0
    
        current_ax.bar(x_0, y_0, color = list_colors[0], \
                                           width = 0.6, edgecolor = 'k')

        current_ax.bar(x_1, y_1, color = list_colors[1], \
                                           width = 0.6, edgecolor = 'k')
    
        current_ax.text(x_0, y_0, str(denominator_0), horizontalalignment = 'center', fontsize = 7)
        current_ax.text(x_1, y_1, str(denominator_1), horizontalalignment = 'center', fontsize = 7)        
    
        xticks.append(p)
        current_tick = age_ticks[age_idx]
        xticklabels.append(current_tick)
        
        p += 2
    
    current_ax.set_yticks([0, 0.5,  1])
    current_ax.set_xticks(xticks)
    current_ax.set_xticklabels(xticklabels)    
    current_ax.spines['right'].set_visible(False)
    current_ax.spines['top'].set_visible(False)    

    current_ax.set_title(list_countries[country_idx])
    current_ax.set_xlabel('Age (in years)', fontweight = 'bold')

    current_ax.set_ylabel('Proportion', fontweight = 'bold')
    
Figure_B.tight_layout()


In [ ]:
Figure_C = plt.figure(figsize = (13, 10))

current_variable = ['outcome_14_num']
print('This is the current variable: ', current_variable)
print('\n')

print('Ratio between heights of the two rows of figures')
heights = [1, 1]
print(heights)

print('Creating figure with add_gridspec')
gs = Figure_C.add_gridspec(2, 1, height_ratios=heights)

print('The second gripspec has 3 columns')
gs0 = gs[0].subgridspec(1, 1)
gs1 = gs[1].subgridspec(1, 3)

print('\n')
print('Size of marker and of font: ')
markersize_list = [10, 5]
fontsizeplot_list = [8, 8]
print(markersize_list)
print(fontsizeplot_list)
print('\n')

print('List of colors and age groups')
list_colors = ['lightskyblue', 'indianred']
age_ranges = [[-1, 120], [0, 18], [18, 60], [60, 120]]
title_list = ['All ages', '< 18 years', '18 - 60 years', ' > 60 years']
print(list_colors)
print(age_ranges)
print(title_list)
print('\n')

for age_index, age_range in enumerate(age_ranges): 

    low_age = age_range[0]
    print(low_age)    
    up_age = age_range[1]
    print(up_age)    
    
    print('Age range: ', low_age, ',', up_age)
    
    temp_data_age = data_Omicron.loc[(data_Omicron['age'] > low_age) & \
                                     (data_Omicron['age'] <= up_age), : ].copy()

    print('Number of observations in the age group: ', temp_data_age.shape[0])
    
    if (age_index == 0):
        current_ax =  Figure_C.add_subplot(gs0[0, 0])
        fontsizeplot = fontsizeplot_list[0]
        markersize = markersize_list[0]
        print('gs0')
        print(age_index)        
        
    else:
        current_ax =  Figure_C.add_subplot(gs1[0, age_index-1])    
        fontsizeplot = fontsizeplot_list[1]        
        markersize = markersize_list[1]        

        print(age_range, 'gs1')
        print(age_index)       
    
    xticks = []
    xticklabels = []

    p = 1
    for country in sorted(list(data_Omicron['iso3'].unique())):
        print(country)
        
        print('Creating age-specific, country-specific, and time period-specific datasets')

        temp_data_0 = temp_data_age.loc[(temp_data_age['iso3'] == country) & \
                                        (temp_data_age['time_periods'] == 0), : ].copy()
        
        temp_data_1 = temp_data_age.loc[(temp_data_age['iso3'] == country) & \
                                        (temp_data_age['time_periods'] == 1) , : ].copy()
        
        
        if len(temp_data_0[current_variable].notnull()) > 0:
            n_0 = np.sum(temp_data_0[current_variable].notnull())[0]

        else:
            n_0 = 0
            
        if len(temp_data_1[current_variable].notnull()) > 0:            
            n_1 = np.sum(temp_data_1[current_variable].notnull())[0]
            
        else :
            n_1 = 0
            
        print('Number of observations in the pre-Omicron and with non-missing outcome: ', temp_data_0.shape[0], n_0)
        print('Number of observations in the Omicron and with non-missing outcome: ', temp_data_1.shape[0], n_1)
            
            
        if (n_0 <10) or (n_1 <10):
            print('Continue since less than 10 observations either in the pre-Omicron or Omicron periods')
            continue

        # X data
        x_0 = p - 0.5
        x_1 = p + 0.5    

        print('\n')        
        print('Creating datasets that include information on the outcome (age-specific, country-specific):')

        temp_data_0_var = pd.DataFrame(temp_data_0[current_variable].value_counts(normalize = True)).sort_index()
        temp_data_1_var = pd.DataFrame(temp_data_1[current_variable].value_counts(normalize = True)).sort_index()    

        
        if 1 in temp_data_0_var.index:
            y_var_0 = temp_data_0_var.loc[1].values[0]
            
        else:
            y_var_0 = 0

        if 1 in temp_data_1_var.index:            
            y_var_1 = temp_data_1_var.loc[1].values[0]
            
        else:
            y_var_1 = 0
            

        current_ax.plot([x_0, x_1], [y_var_0, y_var_1], color = 'k', marker = '.', linewidth = 1)
        current_ax.plot(x_0, y_var_0, color = list_colors[0], marker = 'o', markeredgecolor = 'k', markersize = markersize)
        current_ax.plot(x_1, y_var_1, color = list_colors[1], marker = 'o', markeredgecolor = 'k', markersize = markersize)    

        total_0_var = pd.DataFrame(temp_data_0[current_variable].value_counts()).sort_index().sum()[0]
        total_1_var = pd.DataFrame(temp_data_1[current_variable].value_counts()).sort_index().sum()[0]   

        if 1 in temp_data_0_var.index:
            num_var_0 = pd.DataFrame(temp_data_0[current_variable].value_counts()).sort_index().loc[1].values[0]
        else:
            num_var_0 = 0

        if 1 in temp_data_1_var.index:            
            num_var_1 = pd.DataFrame(temp_data_1[current_variable].value_counts()).sort_index().loc[1].values[0]
        else:
            num_var_1 = 0        

        if total_0_var >= 5:
            lower_0_limit, upper_0_limit = proportion_confint(num_var_0, total_0_var, alpha=0.05, method='beta')
            current_ax.plot([x_0, x_0], [lower_0_limit, upper_0_limit], color = 'k', linewidth = 0.5)
        if total_1_var >= 5:
            lower_1_limit, upper_1_limit = proportion_confint(num_var_1, total_1_var, alpha=0.05, method='beta')
            current_ax.plot([x_1, x_1], [lower_1_limit, upper_1_limit], color = 'k', linewidth = 0.5)        

        print('pre-Omicron')
        print(temp_data_0_var)
        print(y_var_0)
        print(num_var_0, total_0_var)
        print('Omicron')      
        print(temp_data_1_var)        
        print(y_var_1)        
        print(num_var_1, total_1_var)        
        print('\n')     
            
        xticks.append(p)
        xticklabels.append(country)    

        p += 3

    current_ax.set_xticks(xticks)
    current_ax.set_xticklabels(xticklabels, fontsize = fontsizeplot)
    current_ax.set_yticks([0, 0.3, 0.6, 0.9])
    current_ax.set_ylim([-.05, 1.1])        
    current_ax.set_yticklabels([0, 0.3, 0.6, 0.9], fontsize = fontsizeplot) 
    current_ax.set_title(title_list[age_index], fontsize = 10, fontweight = 'bold')

    if age_index < 2:
        current_ax.set_ylabel('Proportion')
    
Figure_C.tight_layout()
                      